In [2]:
import sys
import os
import torch
import matplotlib.pyplot as plt
from monai.inferers import sliding_window_inference
from monai.networks.nets import UNet

sys.path.append('..') 
from configs.config import config
from src.dataset import get_loaders1

In [ ]:
# 1. 设置
stage = 'liver' # 想看谁就改谁
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 2. 准备数据 (只拿验证集)
# 这里只要 val_loader
_, val_loader = get_loaders(Config.DATA_DIR, stage=stage, batch_size=1, cache=False)

# 3. 加载模型结构
model = UNet(
    spatial_dims=3,
    in_channels=1,
    out_channels=2,
    channels=(16, 32, 64, 128, 256),
    strides=(2, 2, 2, 2),
    num_res_units=2,
    norm="batch",
).to(device)

# 4. 加载训练好的权重
model_path = os.path.join('../output/models', f"best_model_{stage}.pth")
if os.path.exists(model_path):
    model.load_state_dict(torch.load(model_path))
    print("成功加载模型权重")
else:
    print(f"找不到模型: {model_path}")

### 推理并画图

In [ ]:
model.eval()
with torch.no_grad():
    # 拿第一个病人来看看
    for i, val_data in enumerate(val_loader):
        images = val_data["image"].to(device)
        labels = val_data["label"].to(device)
        
        # 滑动窗口推理
        val_outputs = sliding_window_inference(
            inputs=images, 
            roi_size=Config.PATCH_SIZE, 
            sw_batch_size=4, 
            predictor=model
        )
        
        # 变成 0/1 结果
        preds = torch.argmax(val_outputs, dim=1).detach().cpu()
        labels = labels.cpu()
        images = images.cpu()
        
        # 画图：找一个有内容的切片 (比如第 80 层，或者用我们之前的自动寻找算法)
        slice_idx = 80 
        
        plt.figure("check", (18, 6))
        plt.subplot(1, 3, 1)
        plt.title(f"Image")
        plt.imshow(images[0, 0, :, :, slice_idx], cmap="gray")
        
        plt.subplot(1, 3, 2)
        plt.title(f"Ground Truth")
        plt.imshow(labels[0, 0, :, :, slice_idx], cmap="jet")
        
        plt.subplot(1, 3, 3)
        plt.title(f"Prediction")
        plt.imshow(preds[0, :, :, slice_idx], cmap="jet")
        
        plt.show()
        
        if i == 0: break # 看一个就够了，想看更多就把这行注释掉